# Auto Crypto Bot

In [1]:
import requests
import pandas as pd
import time

In [3]:
pip install krakenex

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Keys  
# only hard code for example. save in a .txt file for more security 

api_key = "d+3lyALxPCMoRCZVWsWzPUe+j6NC4CuL6OnuReECKzS02SwNvytVaI6n"
api_secret = "1VnV+eFrEdaAfEFyz4JBz01URUo7Tu2ZKJ/N/fBmKCk8O+bDdVihaF4TYSlkto/k+eTGYAVG2V+auNL2/4ExaQ=="


In [ ]:
!pip install yfinance


In [ ]:
!pip install tensorflow

In [5]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.19.0


In [7]:
import krakenex
import yfinance as yf
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

# Hardcode your API key and secret (replace with your actual keys)
api_key = "d+3lyALxPCMoRCZVWsWzPUe+j6NC4CuL6OnuReECKzS02SwNvytVaI6n"
api_secret = "1VnV+eFrEdaAfEFyz4JBz01URUo7Tu2ZKJ/N/fBmKCk8O+bDdVihaF4TYSlkto/k+eTGYAVG2V+auNL2/4ExaQ=="


# Initialize Kraken API client
k = krakenex.API(key=api_key, secret=api_secret)

# Define trading pair and interval
pair = 'XXBTZUSD'  # Bitcoin/USD (Kraken uses 'XBT' for Bitcoin)
interval = 1     # 1-minute OHLC data (adjustable later)

In [9]:
def check_balance():
    """Check account balance with detailed error reporting."""
    try:
        balance = k.query_private('Balance')
        print("Raw API response:", balance)  # Show the full response for debugging
        if balance['error']:
            print(f"API Error: {balance['error']}")
            return None
        return balance['result']
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

# Test it
balance = check_balance()
if balance:
    print("Account Balance:", balance)
else:
    print("Failed to retrieve balance.")

Raw API response: {'error': [], 'result': {'ZUSD': '100.0000'}}
Account Balance: {'ZUSD': '100.0000'}


In [11]:
# Define the ticker and period
ticker = "BTC-USD"
period = "7d"  # Max for 1-minute data in yfinance
interval = "1m"

# Fetch data
df_historical = yf.download(tickers=ticker, period=period, interval=interval)

# Check the data
print(f"Data shape: {df_historical.shape}")
print(f"Time range: {df_historical.index.min()} to {df_historical.index.max()}")
display(df_historical.tail())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Data shape: (5958, 5)
Time range: 2025-04-03 00:01:00+00:00 to 2025-04-09 00:45:00+00:00


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Datetime,,,,,
2025-04-09 00:39:00+00:00,76494.757812,76494.757812,76494.757812,76494.757812,109944832
2025-04-09 00:40:00+00:00,76479.468750,76479.468750,76479.468750,76479.468750,146653184
2025-04-09 00:42:00+00:00,76384.304688,76384.304688,76384.304688,76384.304688,0
2025-04-09 00:44:00+00:00,76517.421875,76517.421875,76517.421875,76517.421875,0
2025-04-09 00:45:00+00:00,76584.359375,76584.359375,76584.359375,76584.359375,204001280


In [13]:
df_historical.shape

(5958, 5)

# Function for retrieving recent BTC DATA

In [16]:
import yfinance as yf
from IPython.display import display

def fetch_recent_btc_data(ticker="BTC-USD", period="7d", interval="1m"):
    """
    Fetches recent historical price data for a given ticker using yfinance.
    
    Parameters:
        ticker (str): The ticker symbol to fetch (default is "BTC-USD").
        period (str): The period of historical data (default is "7d").
        interval (str): The data interval (default is "1m").
        
    Returns:
        DataFrame: Historical price data.
    """
    df = yf.download(tickers=ticker, period=period, interval=interval)
    
    print(f"Data shape: {df.shape}")
    print(f"Time range: {df.index.min()} to {df.index.max()}")
    display(df.tail())
    
    return df


In [18]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Use Close price
data = df_historical['Close'].values.reshape(-1, 1)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences (e.g., 60 minutes lookback)
lookback = 60
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)

# Reshape for LSTM [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split into train and test (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

Training samples: 4718, Test samples: 1180


# Function to Pre-process data:

In [20]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(df, column='Close', lookback=60, train_split=0.8):
    """
    Prepares data for LSTM model from a DataFrame.
    
    Parameters:
        df (DataFrame): Data containing the price info.
        column (str): Column to use for prediction (default 'Close').
        lookback (int): Number of time steps to look back for sequences.
        train_split (float): Proportion of data to use for training.
    
    Returns:
        Tuple: X_train, X_test, y_train, y_test, scaler
    """
    # Extract and scale the specified column
    data = df[column].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Create sequences
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)

    # Reshape for LSTM
    X = X.reshape((X.shape[0], X.shape[1], 1))

    # Train/test split
    train_size = int(len(X) * train_split)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")
    return X_train, X_test, y_train, y_test, scaler


In [22]:
# Define the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(lookback, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile and train
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate on test set
test_loss = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")

Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0740
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0051
Epoch 3/10
 43/148 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0050

KeyboardInterrupt: 

In [26]:
# Function for Model:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_and_train_lstm(X_train, y_train, X_test, y_test, lookback=60, 
                         units=50, dropout=0.2, epochs=10, batch_size=32):
    """
    Builds, trains, and evaluates an LSTM model.

    Parameters:
        X_train, y_train: Training data
        X_test, y_test: Testing data
        lookback (int): Lookback window size
        units (int): LSTM units
        dropout (float): Dropout rate
        epochs (int): Number of training epochs
        batch_size (int): Batch size

    Returns:
        model: Trained LSTM model
    """
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(lookback, 1)))
    model.add(Dropout(dropout))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    print(f"Test loss: {test_loss}")

    return model


# Get Price Function:

In [29]:
import krakenex
from datetime import datetime

# Initialize Kraken API
k = krakenex.API(key="your_api_key_here", secret="your_api_secret_here")
pair = "XXBTZUSD"

def get_current_price(pair):
    """Fetch current price from Kraken."""
    try:
        response = k.query_public('Ticker', {'pair': pair})
        if response['error']:
            print(f"Error: {response['error']}")
            return None
        price = float(response['result'][pair]['c'][0])  # Last trade price
        return price
    except Exception as e:
        print(f"Exception: {e}")
        return None

# Test it
current_price = get_current_price(pair)
if current_price:
    print(f"{datetime.now()}: Current BTC price: {current_price}")
else:
    print("Failed to fetch current price.")

2025-04-08 20:50:48.757466: Current BTC price: 76517.1


# Live Forcasting Function:

In [32]:
import time
import numpy as np
from datetime import datetime

def live_price_forecast(model, scaler, pair, lookback=60, sleep_time=60, get_price_fn=None, initial_data=None):
    """
    Runs a live LSTM-based price forecast loop using live Kraken data.

    Parameters:
        model: Trained LSTM model.
        scaler: Fitted MinMaxScaler used during training.
        pair (str): Kraken pair (e.g., "XXBTZUSD").
        lookback (int): Number of minutes for rolling window input.
        sleep_time (int): Seconds to wait between predictions (default 60).
        get_price_fn (function): Function to fetch live price.
        initial_data (array): Initial price data (e.g., last 60 closes) to seed the model.

    Returns:
        None
    """
    if get_price_fn is None:
        raise ValueError("Please pass a valid function to get_price_fn to fetch live prices.")

    if initial_data is None or len(initial_data) < lookback:
        raise ValueError("Initial data must be provided with at least `lookback` number of values.")

    print("Starting live forecast...")

    # Prepare initial window
    window = np.array(initial_data[-lookback:]).reshape(-1, 1)
    window_scaled = scaler.transform(window)

    while True:
        # Predict next price
        window_input = window_scaled.reshape((1, lookback, 1))
        next_pred_scaled = model.predict(window_input, verbose=0)
        next_pred = scaler.inverse_transform(next_pred_scaled)[0, 0]

        # Get live price
        current_price = get_price_fn(pair)
        if current_price is None:
            print("Kraken API failed, retrying in 60 seconds...")
            time.sleep(sleep_time)
            continue

        # Print forecast info
        print(f"{datetime.now()}: Current Price: {current_price:.2f}, Predicted Next Price: {next_pred:.2f}")

        # Update rolling window
        window = np.append(window[1:], [[current_price]], axis=0)
        window_scaled = scaler.transform(window)

        time.sleep(sleep_time)


# Trading Logic

In [35]:
def adaptive_trading_strategy(current_price, predicted_price, recent_preds, state, volatility, base_threshold=0.005):
    """
    A smarter trading strategy that accounts for prediction direction, consistency, and volatility.

    Parameters:
        current_price (float)
        predicted_price (float)
        recent_preds (list): List of previous predictions
        state (dict)
        volatility (float): Rolling std dev of recent prices
        base_threshold (float): Base percentage move to trigger

    Returns:
        str: "buy", "sell", "hold"
    """
    # Trend direction and consistency
    recent_trend = np.mean(np.diff(recent_preds[-3:])) if len(recent_preds) >= 3 else 0
    pct_diff = (predicted_price - current_price) / current_price
    
    # Adjust threshold dynamically based on volatility
    threshold = base_threshold + (volatility * 0.5)  # more volatile? require bigger moves

    if pct_diff >= threshold and recent_trend > 0 and not state['holding']:
        return "buy"
    elif pct_diff <= -threshold and recent_trend < 0 and state['holding']:
        return "sell"
    else:
        return "hold"


# Adaptive trading Strategy & Forecast

In [37]:
import time
import numpy as np
from datetime import datetime

def adaptive_trading_strategy(current_price, predicted_price, recent_preds, state, volatility, base_threshold=0.005):
    """
    A smarter trading strategy that accounts for prediction direction, consistency, and volatility.

    Parameters:
        current_price (float)
        predicted_price (float)
        recent_preds (list): List of previous predictions
        state (dict)
        volatility (float): Rolling std dev of recent prices
        base_threshold (float): Base percentage move to trigger

    Returns:
        str: "buy", "sell", "hold"
    """
    # Trend direction and consistency
    recent_trend = np.mean(np.diff(recent_preds[-3:])) if len(recent_preds) >= 3 else 0
    pct_diff = (predicted_price - current_price) / current_price
    
    # Adjust threshold dynamically based on volatility
    threshold = base_threshold + (volatility * 0.5)  # more volatile? require bigger moves

    if pct_diff >= threshold and recent_trend > 0 and not state['holding']:
        return "buy"
    elif pct_diff <= -threshold and recent_trend < 0 and state['holding']:
        return "sell"
    else:
        return "hold"

def live_price_forecast_with_trading(model, scaler, pair, lookback=60, sleep_time=60, get_price_fn=None, initial_data=None):
    """
    Runs a live LSTM-based price forecast loop with adaptive trading strategy using live Kraken data.

    Parameters:
        model: Trained LSTM model.
        scaler: Fitted MinMaxScaler used during training.
        pair (str): Kraken pair (e.g., "XXBTZUSD").
        lookback (int): Number of minutes for rolling window input.
        sleep_time (int): Seconds to wait between predictions (default 60).
        get_price_fn (function): Function to fetch live price.
        initial_data (array): Initial price data (e.g., last 60 closes) to seed the model.

    Returns:
        None
    """
    if get_price_fn is None:
        raise ValueError("Please pass a valid function to get_price_fn to fetch live prices.")

    if initial_data is None or len(initial_data) < lookback:
        raise ValueError("Initial data must be provided with at least `lookback` number of values.")

    print("Starting live forecast with trading...")

    # Prepare initial window
    window = np.array(initial_data[-lookback:]).reshape(-1, 1)
    window_scaled = scaler.transform(window)

    # Initialize state and tracking variables
    state = {
        'holding': False,
        'usd_balance': 1000,  # Example starting balance
        'btc_balance': 0,
        'trade_log': []
    }
    recent_preds = []  # Track recent predictions for trend analysis
    price_history = []  # Track recent prices for volatility calculation

    while True:
        # Predict next price
        window_input = window_scaled.reshape((1, lookback, 1))
        next_pred_scaled = model.predict(window_input, verbose=0)
        next_pred = scaler.inverse_transform(next_pred_scaled)[0, 0]

        # Get live price
        current_price = get_price_fn(pair)
        if current_price is None:
            print("Kraken API failed, retrying in 60 seconds...")
            time.sleep(sleep_time)
            continue

        # Add the new prediction to recent_preds and price_history
        recent_preds.append(next_pred)
        price_history.append(current_price)

        # Calculate rolling volatility (standard deviation of last N prices)
        if len(price_history) > 1:
            volatility = np.std(np.diff(price_history[-10:]))  # Last 10 price changes
        else:
            volatility = 0  # If not enough data

        # Run the adaptive strategy to decide if we should buy, sell, or hold
        action = adaptive_trading_strategy(current_price, next_pred, recent_preds, state, volatility)

        if action == "buy":
            # Execute buy: Convert all USD balance into BTC
            btc_bought = state['usd_balance'] / current_price
            state['btc_balance'] = btc_bought
            state['usd_balance'] = 0
            state['holding'] = True
            state['trade_log'].append((datetime.now(), "BUY", current_price))
            print(f"Executed BUY at {current_price:.2f}")
        elif action == "sell":
            # Execute sell: Convert all BTC balance into USD
            usd_gained = state['btc_balance'] * current_price
            state['usd_balance'] = usd_gained
            state['btc_balance'] = 0
            state['holding'] = False
            state['trade_log'].append((datetime.now(), "SELL", current_price))
            print(f"Executed SELL at {current_price:.2f}")
        else:
            print(f"{datetime.now()}: Hold - Current Price: {current_price:.2f}, Predicted Next Price: {next_pred:.2f}")

        # Update rolling window
        window = np.append(window[1:], [[current_price]], axis=0)
        window_scaled = scaler.transform(window)

        # Wait before next iteration
        time.sleep(sleep_time)


In [39]:
initial_window = df_historical['Close'].values  # Or just the last 60 entries
live_price_forecast_with_trading(model, scaler, pair, get_price_fn=get_current_price, initial_data=initial_window)


Starting live forecast with trading...
2025-04-08 20:51:00.825228: Hold - Current Price: 76491.70, Predicted Next Price: 76592.66


KeyboardInterrupt: 

# UI:

In [42]:
!pip install ipywidgets


In [44]:
!pip install ipywidgets widgetsnbextension

In [46]:
!jupyter nbextension enable --py widgetsnbextension


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [48]:
import sys
print(sys.executable)


/opt/anaconda3/bin/python


In [55]:
%pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [50]:
import ipywidgets as widgets
from IPython.display import display

# Widget for adjusting volatility threshold
threshold_slider = widgets.FloatSlider(
    value=0.005,
    min=0.001,
    max=0.05,
    step=0.001,
    description='Threshold:',
    continuous_update=False
)

# Widget for controlling the start/stop of the trading loop
start_button = widgets.Button(description="Start Trading")
stop_button = widgets.Button(description="Stop Trading")

# Output area to display logs
log_output = widgets.Output()

# State to control the loop and other things
loop_running = False
current_state = {
    'threshold': threshold_slider.value
}

# Start trading button callback
def start_trading(b):
    global loop_running
    loop_running = True
    with log_output:
        print(f"Trading started with threshold: {current_state['threshold']}")

    # Run the live loop
    initial_window = df_historical['Close'].values  # Or use last 60 minutes data from Kraken API
    live_price_forecast_with_trading(model, scaler, pair, get_price_fn=get_current_price, initial_data=initial_window)

# Stop trading button callback
def stop_trading(b):
    global loop_running
    loop_running = False
    with log_output:
        print("Trading stopped.")

# Update the threshold value when the slider changes
def on_threshold_change(change):
    current_state['threshold'] = change['new']
    with log_output:
        print(f"Threshold updated to: {current_state['threshold']}")

# Bind events to widgets
threshold_slider.observe(on_threshold_change, names='value')
start_button.on_click(start_trading)
stop_button.on_click(stop_trading)

# Display the widgets
display(threshold_slider, start_button, stop_button, log_output)


FloatSlider(value=0.005, continuous_update=False, description='Threshold:', max=0.05, min=0.001, step=0.001)

Button(description='Start Trading', style=ButtonStyle())

Button(description='Stop Trading', style=ButtonStyle())

Output()

In [3]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread

# Widget for adjusting volatility threshold
threshold_slider = widgets.FloatSlider(
    value=0.005,
    min=0.001,
    max=0.05,
    step=0.001,
    description='Threshold:',
    continuous_update=False
)

# Widget for controlling the trading loop
start_button = widgets.Button(description="Start Trading")
stop_button = widgets.Button(description="Stop Trading")

# Output area to display logs
log_output = widgets.Output()

# State to control the loop and other things
loop_running = False
current_state = {
    'threshold': threshold_slider.value
}

# Start trading button callback
def start_trading(b):
    global loop_running
    if not loop_running:  # Prevent multiple threads
        loop_running = True
        with log_output:
            print(f"Trading started with threshold: {current_state['threshold']}")
        
        # Run the live loop in a separate thread
        initial_window = df_historical['Close'].values
        thread = Thread(target=live_price_forecast_with_trading,
                       args=(model, scaler, pair, get_current_price, initial_window))
        thread.start()

# Stop trading button callback
def stop_trading(b):
    global loop_running
    loop_running = False
    with log_output:
        print("Trading stopped.")

# Update the threshold value when the slider changes
def on_threshold_change(change):
    current_state['threshold'] = change['new']
    with log_output:
        print(f"Threshold updated to: {current_state['threshold']}")

# Bind events to widgets
threshold_slider.observe(on_threshold_change, names='value')
start_button.on_click(start_trading)
stop_button.on_click(stop_trading)

# Display the widgets
display(threshold_slider, start_button, stop_button, log_output)

FloatSlider(value=0.005, continuous_update=False, description='Threshold:', max=0.05, min=0.001, step=0.001)

Button(description='Start Trading', style=ButtonStyle())

Button(description='Stop Trading', style=ButtonStyle())

Output()

In [5]:
import ipywidgets as widgets

In [7]:
widgets.IntSlider()

IntSlider(value=0)

In [9]:
print("Hello")

Hello


In [ ]:

# Initial window (last 60 minutes from historical data)
window = df_historical['Close'].values[-lookback:].reshape(-1, 1)
window_scaled = scaler.transform(window)

# Live forecast loop
print("Starting live forecast...")
while True:
    # Predict next price
    window_input = window_scaled.reshape((1, lookback, 1))
    next_pred_scaled = model.predict(window_input, verbose=0)
    next_pred = scaler.inverse_transform(next_pred_scaled)[0, 0]
    
    # Fetch current price from Kraken
    current_price = get_current_price(pair)
    if current_price is None:
        print("Kraken API failed, retrying in 60 seconds...")
        time.sleep(60)
        continue
    
    # Print results
    print(f"{datetime.now()}: Current Price: {current_price}, Predicted Next Price: {next_pred}")
    
    # Update rolling window
    window = np.append(window[1:], [[current_price]], axis=0)
    window_scaled = scaler.transform(window)
    
    time.sleep(60)  # Wait 1 minute

In [ ]:
# 262.64
# 260.486
# 297.31
# 248.3
#
#

In [ ]:
df_historical.shape

In [ ]:
# Factoring in Day Time. 


# Assuming df_historical is already loaded from yfinance
# Extract hour from index (Yahoo uses datetime index)
df_historical['hour'] = df_historical.index.hour
df_historical['minute'] = df_historical.index.minute  # Optional, for finer granularity

# Cyclical encoding for hour (24-hour cycle)
df_historical['hour_sin'] = np.sin(2 * np.pi * df_historical['hour'] / 24)
df_historical['hour_cos'] = np.cos(2 * np.pi * df_historical['hour'] / 24)

# If using minute (60-minute cycle)
df_historical['minute_sin'] = np.sin(2 * np.pi * df_historical['minute'] / 60)
df_historical['minute_cos'] = np.cos(2 * np.pi * df_historical['minute'] / 60)

# Features for the model: Close price + time features
features = df_historical[['Close', 'hour_sin', 'hour_cos']].values  # Add 'minute_sin', 'minute_cos' if desired

# Scale all features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(features)

# Check the updated data
print(f"Updated shape: {scaled_data.shape}")
display(df_historical.tail())

In [ ]:
lookback = 60
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])  # All features (price + time)
    y.append(scaled_data[i, 0])          # Predict only the Close price
X, y = np.array(X), np.array(y)

# Reshape for LSTM [samples, timesteps, features]
# X shape: (samples, 60, 3) with 3 features
print(f"X shape: {X.shape}")

# Split into train and test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(lookback, X.shape[2])))  # X.shape[2] = 3 features
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))  # Still predict 1 value (Close price)

# Compile and train
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate
test_loss = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")

In [ ]:
import krakenex
import numpy as np
import time
from datetime import datetime

# Kraken API setup
k = krakenex.API(key="your_api_key_here", secret="your_api_secret_here")
pair = "XXBTZUSD"

def get_current_price(pair):
    try:
        response = k.query_public('Ticker', {'pair': pair})
        if response['error']:
            print(f"Error: {response['error']}")
            return None
        return float(response['result'][pair]['c'][0])
    except Exception as e:
        print(f"Exception: {e}")
        return None

# Initial window (last 60 timesteps from historical data)
window = scaled_data[-lookback:]  # Shape: (60, 3)

# Live loop
print("Starting live forecast with time features...")
while True:
    # Predict next price
    window_input = window.reshape((1, lookback, window.shape[1]))
    next_pred_scaled = model.predict(window_input, verbose=0)
    next_pred = scaler.inverse_transform(
        np.array([[next_pred_scaled[0, 0], 0, 0]])  # Dummy values for hour features
    )[0, 0]  # Extract only the price
    
    # Fetch current price and time
    current_price = get_current_price(pair)
    if current_price is None:
        print("Kraken API failed, retrying...")
        time.sleep(60)
        continue
    
    now = datetime.now()
    hour = now.hour
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)
    
    # New data point [price, hour_sin, hour_cos]
    new_point = np.array([[current_price, hour_sin, hour_cos]])
    new_point_scaled = scaler.transform(new_point)
    
    # Update window
    window = np.append(window[1:], new_point_scaled, axis=0)
    
    print(f"{now}: Current Price: {current_price}, Predicted Next Price: {next_pred}")
    time.sleep(60)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features)

lookback = 60  # 60 minutes for training
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(lookback, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

test_loss = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")

In [ ]:
import krakenex
import numpy as np
import time
from datetime import datetime

# Kraken API setup
k = krakenex.API(key="your_api_key_here", secret="your_api_secret_here")
pair = "XXBTZUSD"

def get_current_price(pair, last_price=None):
    """Fetch current price from Kraken, with rate limit handling."""
    global last_fetch_time, last_fetched_price
    current_time = time.time()
    
    # Fetch every 3 seconds to stay under rate limit (~20 calls/min)
    if 'last_fetch_time' not in globals() or (current_time - last_fetch_time) >= 3:
        try:
            response = k.query_public('Ticker', {'pair': pair})
            if response['error']:
                print(f"Error: {response['error']}")
                return last_price  # Reuse last price on error
            last_fetched_price = float(response['result'][pair]['c'][0])
            last_fetch_time = current_time
            return last_fetched_price
        except Exception as e:
            print(f"Exception: {e}")
            return last_price
    return last_fetched_price if 'last_fetched_price' in globals() else last_price

# Initialize window with last 60 minutes (scaled) from training
window = scaled_data[-lookback:]  # Shape: (60, 3), assuming lookback=60
last_price = get_current_price(pair)  # Initial fetch

# Adjust to 60-second window (fill with last known price initially)
window_seconds = np.zeros((60, 3))
window_seconds[:, 0] = last_price  # Fill price
window_seconds[:, 1:] = window[-1, 1:]  # Copy last hour_sin, hour_cos
window_seconds_scaled = scaler.transform(window_seconds)

# Live loop every 10 seconds
print("Starting live forecast every 10 seconds...")
while True:
    start_time = time.time()
    
    # Predict next price (10 seconds ahead)
    window_input = window_seconds_scaled.reshape((1, 60, 3))
    next_pred_scaled = model.predict(window_input, verbose=0)
    next_pred = scaler.inverse_transform(
        np.array([[next_pred_scaled[0, 0], 0, 0]])
    )[0, 0]
    
    # Fetch or reuse current price
    current_price = get_current_price(pair, last_price)
    if current_price is None:
        print("No price available, skipping...")
        time.sleep(10)
        continue
    last_price = current_price
    
    # Update time features
    now = datetime.now()
    hour = now.hour
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)
    new_point = np.array([[current_price, hour_sin, hour_cos]])
    new_point_scaled = scaler.transform(new_point)
    
    # Update 60-second window
    window_seconds_scaled = np.append(window_seconds_scaled[1:], new_point_scaled, axis=0)
    
    print(f"{now}: Current Price: {current_price}, Predicted Next Price (10s ahead): {next_pred}")
    
    # Sleep to maintain 10-second intervals
    elapsed = time.time() - start_time
    if elapsed < 10:
        time.sleep(10 - elapsed)